In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import deepcopy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%time
df = pd.read_csv('/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/bars_sec.csv')
df = df.fillna(0)
df.head(2)

CPU times: user 9.27 s, sys: 970 ms, total: 10.2 s
Wall time: 11.8 s


,datetime,mean_price,bid_0_size,bid_1_size,bid_2_size,bid_3_size,bid_4_size,bid_5_size,bid_6_size,bid_7_size,...,ask_0_size,ask_1_size,ask_2_size,ask_3_size,ask_4_size,ask_5_size,ask_6_size,ask_7_size,ask_8_size,ask_9_size
0,2020-11-01 01:36:15,13763.75,11627297.0,1169276.0,1159108.0,202800.0,123725.0,1804059.0,724937.0,356270.0,...,8753411.0,2082535.0,923013.0,884802.0,2109313.0,1448770.0,7149895.0,4016421.0,1140271.0,1695146.0
1,2020-11-01 01:36:16,13763.75,17887488.0,1820160.0,1814256.0,108000.0,44613.0,3114540.0,1134684.0,557640.0,...,13189142.0,3259620.0,1444716.0,1177848.0,3176316.0,2267640.0,11191140.0,6286572.0,1784772.0,2653272.0


In [3]:
df['diff'] = df['mean_price'].diff()
df = df.dropna()
df.index = range(len(df))
diff = df['diff'].values

target = list()
for i in range(len(df)):
    if diff[i] < 0:
        target.append('Down')
    elif diff[i] > 0:
        target.append('Up')
    else:
        target.append('Unchange')

df['target'] =  target
df = df.drop(columns=['diff'])
target = []

df.head(3)

,datetime,mean_price,bid_0_size,bid_1_size,bid_2_size,bid_3_size,bid_4_size,bid_5_size,bid_6_size,bid_7_size,...,ask_1_size,ask_2_size,ask_3_size,ask_4_size,ask_5_size,ask_6_size,ask_7_size,ask_8_size,ask_9_size,target
0,2020-11-01 01:36:16,13763.75,17887488.0,1820160.0,1814256.0,108000.0,44613.0,3114540.0,1134684.0,557640.0,...,3259620.0,1444716.0,1177848.0,3176316.0,2267640.0,11191140.0,6286572.0,1784772.0,2653272.0,Unchange
1,2020-11-01 01:36:17,13763.75,15967120.0,1617920.0,1612672.0,96000.0,12992.0,3004188.0,1401762.0,495680.0,...,2897440.0,1284192.0,1046976.0,2911208.0,2015680.0,9947680.0,5588064.0,1586464.0,2358464.0,Unchange
2,2020-11-01 01:36:18,13763.75,41788094.0,4145920.0,4132472.0,246000.0,286362.0,8020650.0,4268018.0,1270180.0,...,7003490.0,3290742.0,3344376.0,8222116.0,5165180.0,25490930.0,14319414.0,4065314.0,6043564.0,Unchange


In [4]:
# df = df.rename(columns={"datetime": "datetime", "Mean price": "mid price",
#                         "Bid Size 0": "bid_0_size","Bid Size 1": "bid_1_size","Bid Size 2": "bid_2_size",
#                         "Bid Size 3": "bid_3_size","Bid Size 4": "bid_4_size","Bid Size 5": "bid_5_size",
#                         "Bid Size 6": "bid_6_size","Bid Size 7": "bid_7_size","Bid Size 8": "bid_8_size",
#                         "Bid Size 9": "bid_9_size",
#                         "Ask Size 0": "ask_0_size","Ask Size 1": "ask_1_size","Ask Size 2": "ask_2_size",
#                         "Ask Size 3": "ask_3_size","Ask Size 4": "ask_4_size","Ask Size 5": "ask_5_size",
#                         "Ask Size 6": "ask_6_size","Ask Size 7": "ask_7_size","Ask Size 8": "ask_8_size",
#                         "Ask Size 9": "ask_9_size",
#                        })

In [5]:
df = df.rename(columns={"datetime": "datetime", "mean_price": "mid price",})

In [6]:
%%time
def calculate_imbalance(Sum_bid, Sum_ask):
    imbalance = np.log(Sum_bid)-np.log(Sum_ask)
    return imbalance

def calculate_volume(*args):
    return sum(args)

bid_0,bid_1,bid_2,bid_3,bid_4 = df['bid_0_size'].values,df['bid_1_size'].values,df['bid_2_size'].values,df['bid_3_size'].values,df['bid_4_size'].values
bid_5,bid_6,bid_7,bid_8,bid_9 = df['bid_5_size'].values,df['bid_6_size'].values,df['bid_7_size'].values,df['bid_8_size'].values,df['bid_9_size'].values

ask_0,ask_1,ask_2,ask_3,ask_4 = df['ask_0_size'].values,df['ask_1_size'].values,df['ask_2_size'].values,df['ask_3_size'].values,df['ask_4_size'].values
ask_5,ask_6,ask_7,ask_8,ask_9 = df['ask_5_size'].values,df['ask_6_size'].values,df['ask_7_size'].values,df['ask_8_size'].values,df['ask_9_size'].values

CPU times: user 688 µs, sys: 0 ns, total: 688 µs
Wall time: 656 µs


In [7]:
df['target'].value_counts(normalize=True)*100

Unchange    94.568395
Up           2.747060
Down         2.684545
Name: target, dtype: float64

In [8]:
df_imbalance = df[['datetime','mid price','target']]
df_imbalance['Imb_1'] = calculate_imbalance(bid_0,ask_0)

df_imbalance['Imb_3'] = calculate_imbalance(calculate_volume(bid_0,bid_1,bid_2,),
                                            calculate_volume(ask_0,ask_1,ask_2,))

df_imbalance['Imb_5'] = calculate_imbalance(calculate_volume(bid_0,bid_1,bid_2,bid_3,bid_4),
                                            calculate_volume(ask_0,ask_1,ask_2,ask_3,ask_4))

df_imbalance['Imb_7'] = calculate_imbalance(calculate_volume(bid_0,bid_1,bid_2,bid_3,bid_4,bid_5,bid_6),
                                            calculate_volume(ask_0,ask_1,ask_2,ask_3,ask_4,ask_5,ask_6))

df_imbalance['Imb_10'] = calculate_imbalance(calculate_volume(bid_0,bid_1,bid_2,bid_3,bid_4,bid_5,bid_6,bid_7,bid_8,bid_9,),
                                            calculate_volume(ask_0,ask_1,ask_2,ask_3,ask_4,ask_5,ask_6,ask_7,ask_8,ask_9,))

<timed exec>:2: RuntimeWarning: divide by zero encountered in log
<timed exec>:2: RuntimeWarning: invalid value encountered in subtract
<ipython-input-8-0b27c0a6229d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imbalance['Imb_1'] = calculate_imbalance(bid_0,ask_0)
<ipython-input-8-0b27c0a6229d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imbalance['Imb_3'] = calculate_imbalance(calculate_volume(bid_0,bid_1,bid_2,),
<ipython-input-8-0b27c0a6229d>:7: SettingWithCopyWarning: 
A value is trying 

In [9]:
df.shape
df.dropna().shape

(6259310, 23)

(6259310, 23)

In [10]:
df = df.fillna(0)
df = df_imbalance
df.head(3)

,datetime,mid price,target,Imb_1,Imb_3,Imb_5,Imb_7,Imb_10
0,2020-11-01 01:36:16,13763.75,Unchange,0.304708,0.184635,-0.026099,-0.320171,-0.524092
1,2020-11-01 01:36:17,13763.75,Unchange,0.309791,0.188779,-0.027918,-0.293981,-0.498950
2,2020-11-01 01:36:18,13763.75,Unchange,0.197924,0.116100,-0.104009,-0.322272,-0.515018


In [11]:
df['target'].value_counts(normalize=True)*100

Unchange    94.568395
Up           2.747060
Down         2.684545
Name: target, dtype: float64

In [12]:
df['Imb_7'].min()
df['Imb_7'].max()

-7.86563961124752

7.643481665082431

In [14]:
df_test = df[df['target']!='Unchange']
df_test['target'].value_counts(normalize=True)*100
df_test[(df_test['Imb_7']>-8) & (df_test['Imb_7']<=-2.522348)]['target'].value_counts(normalize=True)*100
df_test[(df_test['Imb_7']>=2.522348) & (df_test['Imb_7']<8)]['target'].value_counts(normalize=True)*100

Up      50.575473
Down    49.424527
Name: target, dtype: float64

Down    97.741503
Up       2.258497
Name: target, dtype: float64

Up      98.085955
Down     1.914045
Name: target, dtype: float64

In [15]:
len(df_test)
len(df_test[(df_test['Imb_7']>-8) & (df_test['Imb_7']<=-2.522348)])
len(df_test[(df_test['Imb_7']>=2.522348) & (df_test['Imb_7']<8)])

339981

41045

50887

In [16]:
df_result = deepcopy(df[['datetime']])

In [17]:
%%time
list_label = list()
imbalance = df['Imb_7'].values

for i in range(len(df)):
    if -8 < imbalance[i] <= -2.522348:
        list_label.append('Down')
    elif 2.522348 <= imbalance[i] < 8:
        list_label.append('Up')       
    else:
        list_label.append('Unchanged')
df_result['Labels 7'] = list_label

CPU times: user 4.37 s, sys: 0 ns, total: 4.37 s
Wall time: 4.37 s


In [18]:
df_result.head(2)

,datetime,Labels 7
0,2020-11-01 01:36:16,Unchanged
1,2020-11-01 01:36:17,Unchanged


In [21]:
df_result.to_csv('/home/vlad/HDD/GIT/CryptoLp DataScience/Projects/05. Imbalanced/IMBALANCES.csv',index=False)

In [19]:
df_result['Labels 7'].value_counts(normalize=True)*100

Unchanged    86.058208
Up            7.904673
Down          6.037119
Name: Labels 7, dtype: float64